# 51515 Medium Angular Motors System Identification

The following notebook handles with system identification of the 51515, Lego Mindstorm, Robot Inventor, Medium Angular Motors.


## DC motor model

The follow [DC motor model](https://ctms.engin.umich.edu/CTMS/index.php?example=MotorSpeed&section=SystemModeling) will be used and the approach described in [University of Utah MEC lab excercise](https://my.mech.utah.edu/~me3200/labs/S04Labs/S04_MotorChar_L04.pdf) will be used for estimating system parameters. Data will be used from [Measured data](https://www.philohome.com/motors/motorcomp.htm) and [Lego](https://le-www-live-s.legocdn.com/sc/media/files/support/spike-prime/techspecs_technicmediumangularmotor-19684ffc443792280359ef217512a1d1.pdf).


#### Internal resistance and back EMF constant

During steady state, the electrical equation of a DC motor simplifies to:

\begin{equation}
V = I R + K_e \omega 
\end{equation}

Combining this with [Loaded characteristics; measured data](https://www.philohome.com/motors/motorcomp.htm) it is possible to use linear regression for estimating $R$ and $K_e$.



In [1]:
import numpy as np
from helper_functions import *

V = np.array([4.5, 6.0, 7.5, 9.0, 10.5, 12.0])
I = np.array([0.29, 0.28, 0.29, 0.30, 0.31, 0.32])
rpm = np.array([24.0, 63.0, 105.0, 138.0, 180.0, 213.0])
omega = rpm_to_rad_per_s(rpm)

results = ols_voltage_current_rotations(V, I, omega)

print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          2.152e+04
Date:                Sun, 05 Mar 2023   Prob (F-statistic):                    8.64e-09
Time:                        09:13:11   Log-Likelihood:                          6.3995
No. Observations:                   6   AIC:                                     -8.799
Df Residuals:                       4   BIC:                                     -9.215
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

c:\repos\51515-System-Identification\51515-System-Identification\.venv\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [2]:
import plotly.express as px

px.line(x=V, y=rpm, markers=True)

## Compare this estimate with estimate based on measurements

In [3]:
from paths import test_results_folder
import pandas as pd

data = pd.read_pickle(test_results_folder.joinpath('2023_03_03_Motor_1_Program_1.pkl'))

ss_time = [(0,800), (1000, 1800), (2000, 2800), (3000, 3800), (4000, 4800), (5000, 5800)]

V, I, omega, rpm = select_data_subset(data=data, ss_timestamps=ss_time)

print(V)
print(I)
print(omega)

X = np.hstack((I.reshape((6,1)), omega.reshape((6,1))))
y = V

results = sm.OLS(y, X).fit()
print(results.summary())

[0.         1.64006278 3.28353459 4.92821346 6.57247587 8.21689539]
[0.         0.0299446  0.04561127 0.06279105 0.08280478 0.08587594]
[ 0.          2.35381945  6.29057191 10.55353316 14.83447065 19.52387183]
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                              6705.
Date:                Sat, 04 Mar 2023   Prob (F-statistic):                    8.89e-08
Time:                        20:46:56   Log-Likelihood:                          6.2138
No. Observations:                   6   AIC:                                     -8.428
Df Residuals:                       4   BIC:                                     -8.844
Df Model:                           2                                                 

c:\repos\51515-System-Identification\51515-System-Identification\.venv\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning:

omni_normtest is not valid with less than 8 observations; 6 samples were given.



In [4]:
px.line(x=V, y=rpm, markers=True)